In [1]:
import hazelcast
import logging
logging.basicConfig(level=logging.INFO)
from hazelcast import HazelcastClient
import json
import time
import gzip

# Create client connection

In [5]:
# Create client
client = hazelcast.HazelcastClient(
    cluster_members=["127.0.0.1:5701", "127.0.0.1:5702", ])

# Connect to the Hazelcast cluster
client = HazelcastClient()

INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is STARTING
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is STARTED
INFO:hazelcast.connection:Trying to connect to Address(host=127.0.0.1, port=5701)
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is CONNECTED
INFO:hazelcast.connection:Authenticated with server Address(host=127.0.0.1, port=5701):64d6a794-babf-4b5e-9798-90b31ccf0584, server version: 5.2.4, local address: Address(host=127.0.0.1, port=54417)
INFO:hazelcast.cluster:

Members [1] {
	Member [127.0.0.1]:5701 - 64d6a794-babf-4b5e-9798-90b31ccf0584
}

INFO:hazelcast.client:Client started


# Specify map

In [ ]:
map_name = "smart-meter-benchmark"
my_map = client.get_map(map_name)

# Data Compression

In [61]:
def compress_csv_file(input_file, output_file):
    with open(input_file, 'rb') as f_in, gzip.open(output_file, 'wb') as f_out:
        f_out.writelines(f_in)

input_file = 'daily_dataset_small.json'
output_file = 'daily_dataset_small.gz'
compress_csv_file(input_file, output_file)

# Data Insertion

In [65]:
# Specify path
gzip_file_path = "/Users/jafarabdurrahmaan/Dropbox/00_Uni/Masterthesis/A_Thesis/1_Components/13_Benchmark_Study_Components/3_KVS/Hazelcast/iter2/daily_dataset_small.gz"

# Start measuring time
start_time = time.time()

# Insert data
with gzip.open(gzip_file_path, "rt") as file:
    file_content = file.read()
    file_content = file_content.rstrip(',')
    json_data = eval(file_content)
    for item in json_data:
        key = item[0]
        value = item[1]
        my_map.put(key, value)

# Stop measuring time
end_time = time.time()

# Calculate ingestion time
ingestion_time = end_time - start_time

# Print ingestion time
print("Ingestion time:", ingestion_time, "seconds")

# Close client
client.shutdown()


INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is STARTING
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is STARTED
INFO:hazelcast.connection:Trying to connect to Address(host=127.0.0.1, port=5701)
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is CONNECTED
INFO:hazelcast.connection:Authenticated with server Address(host=127.0.0.1, port=5701):64d6a794-babf-4b5e-9798-90b31ccf0584, server version: 5.2.4, local address: Address(host=127.0.0.1, port=58911)
INFO:hazelcast.cluster:

Members [1] {
	Member [127.0.0.1]:5701 - 64d6a794-babf-4b5e-9798-90b31ccf0584
}

INFO:hazelcast.client:Client started
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is SHUTTING_DOWN
INFO:hazelcast.connection:Removed connection to Address(host=127.0.0.1, port=5701):64d6a794-babf-4b5e-9798-90b31ccf0584, connection: Connection(id=0, live=False, remote_address=Address(host=127.0.0.1, port=5701))
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is DISCONNECTED
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is SHUTDOWN


Ingestion time: 0.47696471214294434 seconds


# Simple Retrieval

## First Query Workload

In [74]:
# Specify keys
keys = ["MAC000131_2012-06-01", "MAC000131_2012-06-02", "MAC000131_2012-06-03"]

# Create dictionary to store values
values = {}

# Start measuring time
start_time = time.time()

# Retrieve values
for key in keys:
    future = my_map.get(key)
    values[key] = future.result()

# Stop measuring time
end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time

# Print values
for key, value in values.items():
    print(f"Key: {key}, Value: {value}")

# Print execution time
print("Query execution time:", execution_time*1000, "milliseconds")

# Close client
client.shutdown()

INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is STARTING
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is STARTED
INFO:hazelcast.connection:Trying to connect to Address(host=127.0.0.1, port=5701)
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is CONNECTED
INFO:hazelcast.connection:Authenticated with server Address(host=127.0.0.1, port=5701):64d6a794-babf-4b5e-9798-90b31ccf0584, server version: 5.2.4, local address: Address(host=127.0.0.1, port=58987)
INFO:hazelcast.cluster:

Members [1] {
	Member [127.0.0.1]:5701 - 64d6a794-babf-4b5e-9798-90b31ccf0584
}

INFO:hazelcast.client:Client started
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is SHUTTING_DOWN
INFO:hazelcast.connection:Removed connection to Address(host=127.0.0.1, port=5701):64d6a794-babf-4b5e-9798-90b31ccf0584, connection: Connection(id=0, live=False, remote_address=Address(host=127.0.0.1, port=5701))
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is DISCONNECTED
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is SHUTDOWN


Key: MAC000131_2012-06-01, Value: 5.775
Key: MAC000131_2012-06-02, Value: 5.459
Key: MAC000131_2012-06-03, Value: 6.481
Query execution time: 1.1782646179199219 milliseconds


## Second Query Workload

In [71]:
# Specify keys
keys = ["MAC000131_2012-06-01", "MAC000131_2012-06-02", "MAC000131_2012-06-03"]

# Create dictionary to store values
values = {}

# Start measuring time
start_time = time.time()

# Retrieve values
for key in keys:
    future = my_map.get(key)
    values[key] = future.result()

# Stop measuring time
end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time

# Print values
for key, value in values.items():
    print(f"Key: {key}, Value: {value}")

# Print execution time
print("Query execution time:", execution_time*1000, "milliseconds")

# Close client
client.shutdown()

INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is STARTING
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is STARTED
INFO:hazelcast.connection:Trying to connect to Address(host=127.0.0.1, port=5701)
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is CONNECTED
INFO:hazelcast.connection:Authenticated with server Address(host=127.0.0.1, port=5701):64d6a794-babf-4b5e-9798-90b31ccf0584, server version: 5.2.4, local address: Address(host=127.0.0.1, port=58984)
INFO:hazelcast.cluster:

Members [1] {
	Member [127.0.0.1]:5701 - 64d6a794-babf-4b5e-9798-90b31ccf0584
}

INFO:hazelcast.client:Client started
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is SHUTTING_DOWN
INFO:hazelcast.connection:Removed connection to Address(host=127.0.0.1, port=5701):64d6a794-babf-4b5e-9798-90b31ccf0584, connection: Connection(id=0, live=False, remote_address=Address(host=127.0.0.1, port=5701))
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is DISCONNECTED
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is SHUTDOWN


Key: MAC000131_2012-06-01, Value: 5.775
Key: MAC000131_2012-06-02, Value: 5.459
Key: MAC000131_2012-06-03, Value: 6.481
Query execution time: 1.2979507446289062 milliseconds


## Third Query Workload

In [70]:
# Specify keys
keys = ["MAC000131_2012-06-01", "MAC000131_2012-06-02", "MAC000131_2012-06-03"]

# Create dictionary to store values
values = {}

# Start measuring time
start_time = time.time()

# Retrieve values
for key in keys:
    future = my_map.get(key)
    values[key] = future.result()

# Stop measuring time
end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time

# Print values
for key, value in values.items():
    print(f"Key: {key}, Value: {value}")

# Print execution time
print("Query execution time:", execution_time*1000, "milliseconds")

# Close client
client.shutdown()

INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is STARTING
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is STARTED
INFO:hazelcast.connection:Trying to connect to Address(host=127.0.0.1, port=5701)
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is CONNECTED
INFO:hazelcast.connection:Authenticated with server Address(host=127.0.0.1, port=5701):64d6a794-babf-4b5e-9798-90b31ccf0584, server version: 5.2.4, local address: Address(host=127.0.0.1, port=58979)
INFO:hazelcast.cluster:

Members [1] {
	Member [127.0.0.1]:5701 - 64d6a794-babf-4b5e-9798-90b31ccf0584
}

INFO:hazelcast.client:Client started
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is SHUTTING_DOWN
INFO:hazelcast.connection:Removed connection to Address(host=127.0.0.1, port=5701):64d6a794-babf-4b5e-9798-90b31ccf0584, connection: Connection(id=0, live=False, remote_address=Address(host=127.0.0.1, port=5701))
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is DISCONNECTED
INFO:hazelcast.lifecycle:HazelcastClient 5.3.0 is SHUTDOWN


Key: MAC000131_2012-06-01, Value: 5.775
Key: MAC000131_2012-06-02, Value: 5.459
Key: MAC000131_2012-06-03, Value: 6.481
Query execution time: 1.744985580444336 milliseconds


## Overall Query Execution Time for Simple Retrieval

In [75]:
print(f"Average Query Execution Time: {((1.18+1.3+1.75)/3)} milliseconds")

Average Query Execution Time: 1.4100000000000001 milliseconds
